To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

**Read our [blog post](https://unsloth.ai/blog/r1-reasoning) for guidance on how to train reasoning models.**

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
import sys; modules = list(sys.modules.keys())
for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None

%pip install "unsloth==2025.2.4" vllm
%pip install -q --upgrade pillow
%pip install -q transformers==4.31.0
%pip install -q rouge_score bert_score datasets evaluate scikit-learn sentence_transformers sacremoses
# If you are running this notebook on local, you need to install `diffusers` too
%pip install -q diffusers
# Temporarily install a specific TRL nightly version
%pip install -q git+https://github.com/huggingface/trl.git@e95f9fb74a3c3647b86f251b7e230ec51c64b72b


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2025.2.7 requires transformers>=4.46.1, but you have transformers 4.31.0 which is incompatible.
vllm 0.7.3 requires tokenizers>=0.19.1, but you have tokenizers 0.13.3 which is incompatible.
vllm 0.7.3 requires transformers>=4.48.2, but you have transformers 4.31.0 which is incompatible.
unsloth 2025.2.4 requires transformers!=4.47.0,>=4.46.1, but you have transformers 4.31.0 which is incompatible.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

True

### Unsloth

Use `PatchFastRL` before all functions to patch GRPO and other RL algorithms!

In [3]:
from unsloth import FastLanguageModel, PatchFastRL
PatchFastRL("GRPO", FastLanguageModel)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/workspace/qwen_repo/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


2025-02-22 06:31:15,873	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Load up `Qwen 2.5 3B Instruct`, and set parameters

In [4]:
from unsloth import is_bfloat16_supported
import torch
max_seq_length = 1024 # Can increase for longer reasoning traces
lora_rank = 64 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen2.5-3B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.5, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

INFO 02-22 06:31:18 __init__.py:207] Automatically detected platform cuda.
==((====))==  Unsloth 2025.2.4: Fast Qwen2 patching. Transformers: 4.49.0.
   \\   /|    GPU: NVIDIA GeForce RTX 3090. Max memory: 23.57 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen2.5-3b-instruct-unsloth-bnb-4bit with actual GPU utilization = 49.45%
Unsloth: Your GPU has CUDA compute capability 8.6 with VRAM = 23.57 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 512. Num Sequences = 224.
Unsloth: vLLM's KV Cache can use up to 9.24 GB. Also swap space = 6 GB.
INFO 02-22 06:31:26 config.py:549] This model supports multiple tasks: {'reward', 'score', 'classify', 'embed', 'generat

[W222 06:31:28.639190178 CUDAAllocatorConfig.h:28] Warning: expandable_segments not supported on this platform (function operator())


INFO 02-22 06:31:28 weight_utils.py:254] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.21it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.20it/s]

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.03it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.03it/s]



INFO 02-22 06:31:30 model_runner.py:1115] Loading model weights took 2.2160 GB
INFO 02-22 06:31:30 punica_selector.py:18] Using PunicaWrapperGPU.
INFO 02-22 06:31:31 worker.py:267] Memory profiling takes 1.10 seconds
INFO 02-22 06:31:31 worker.py:267] the current vLLM instance can use total_gpu_memory (23.57GiB) x gpu_memory_utilization (0.49) = 11.66GiB
INFO 02-22 06:31:31 worker.py:267] model weights take 2.22GiB; non_torch_memory takes 0.06GiB; PyTorch activation peak memory takes 1.23GiB; the rest of the memory reserved for KV Cache is 8.16GiB.
INFO 02-22 06:31:31 executor_base.py:111] # cuda blocks: 14851, # CPU blocks: 10922
INFO 02-22 06:31:31 executor_base.py:116] Maximum concurrency for 512 tokens per request: 464.09x
INFO 02-22 06:31:37 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error 

Capturing CUDA graph shapes: 100%|██████████| 31/31 [00:21<00:00,  1.42it/s]

INFO 02-22 06:31:59 model_runner.py:1562] Graph capturing finished in 22 secs, took 0.73 GiB
INFO 02-22 06:31:59 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 29.40 seconds



Unsloth 2025.2.4 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


### Data Prep
<a name="Data"></a>

We directly leverage [@willccbb](https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb) for data prep and all reward functions. You are free to create your own!

In [5]:
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split

SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

def get_medical_questions(split="train") -> Dataset:
    # Load the raw dataset from the hub
    data = load_dataset('FreedomIntelligence/medical-o1-reasoning-SFT', 'en')[split]
    # Convert to a pandas DataFrame for easier splitting
    df = data.to_pandas()

    if split == "train":
        # Split the data into 70% train and 30% temporary data
        train_df, temp_df = train_test_split(df, test_size=0.01, random_state=42)
        # Split the temporary data equally into eval and test (each 2.5% of the total) 
        eval_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

        # Convert back to Hugging Face Datasets
        train_dataset = Dataset.from_pandas(train_df)
        eval_dataset = Dataset.from_pandas(eval_df)
        test_dataset = Dataset.from_pandas(test_df)

        # Define a mapping function to create the prompt and answer fields
        def map_fn(x):
            return {
                'prompt': [
                    {'role': 'system', 'content': SYSTEM_PROMPT},
                    {'role': 'user', 'content': x['Question']}
                ],
                'answer': x['Response'],  # The reference answer
                'question': x['Question']
            }

        # Apply the mapping function to each split and remove unnecessary columns
        train_dataset = train_dataset.map(map_fn).remove_columns(['Question', 'Complex_CoT', 'Response', '__index_level_0__'])
        eval_dataset = eval_dataset.map(map_fn).remove_columns(['Question', 'Complex_CoT', 'Response', '__index_level_0__'])
        test_dataset = test_dataset.map(map_fn).remove_columns(['Question', 'Complex_CoT', 'Response', '__index_level_0__'])

        return train_dataset, eval_dataset, test_dataset  # Return all three splits
    else:
        # For non-"train" splits, just map over the data and remove unnecessary columns
        data = data.map(lambda x: {
            'prompt': [
                {'role': 'system', 'content': SYSTEM_PROMPT},
                {'role': 'user', 'content': x['Question']}
            ],
            'answer': x['Response'],
            'question': x['Question']
        }).remove_columns(['Question', 'Complex_CoT', 'Response', '__index_level_0__'])
        return data

# To get the three splits, call:
train_dataset, eval_dataset, test_dataset = get_medical_questions(split="train")
# train_dataset = train_dataset.select(range(3)), 
train_dataset, eval_dataset, test_dataset

Map: 100%|██████████| 127/127 [00:00<00:00, 16953.42 examples/s]


(Dataset({
     features: ['prompt', 'answer', 'question'],
     num_rows: 25117
 }),
 Dataset({
     features: ['prompt', 'answer', 'question'],
     num_rows: 127
 }),
 Dataset({
     features: ['prompt', 'answer', 'question'],
     num_rows: 127
 }))

In [6]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import CrossEncoder
from typing import List
import re

# ------ Device Configuration ------
main_device = "cuda" if torch.cuda.is_available() else "cpu"
reward_device = "cuda" if torch.cuda.is_available() else "cpu"

# ------ Semantic Correctness Reward ------
def semantic_correctness(responses: List[str], answers: List[str]) -> List[float]:
    """Calculate semantic similarity using cross-encoder"""
    model = CrossEncoder('cross-encoder/stsb-roberta-base', device=reward_device)
    with torch.no_grad():            
        inputs = list(zip(responses, answers))
        similarities = model.predict(inputs, show_progress_bar=False).tolist()
        # Set similarity to -1 if the response is an empty string
        similarities = [-1.0 if response == "" else similarity for response, similarity in zip(responses, similarities)]
        return similarities

# ------ Efficient Perplexity Calculation ------
class PerplexityCalculator:
    def __init__(self, model_name="microsoft/biogpt", device=reward_device):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.device = device
        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.model = AutoModelForCausalLM.from_pretrained(model_name).to(self.device)
        self.model.eval()

    def calculate(self, texts: List[str], batch_size=8) -> List[float]:
        perplexities = []
        for i in range(0, len(texts), batch_size):
            batch = texts[i:i+batch_size]
            try:
                if not batch: continue
                
                encodings = self.tokenizer(
                    batch, 
                    return_tensors="pt", 
                    padding=True, 
                    truncation=True,
                    max_length=200
                ).to(self.device)
                
                with torch.no_grad():
                    outputs = self.model(**encodings, labels=encodings.input_ids)
                
                loss = outputs.loss
                if torch.isnan(loss):
                    raise ValueError("NaN loss encountered")
                
                batch_perplexity = torch.exp(loss).repeat(len(batch)).cpu().tolist()
                # Set perplexity to -1 if the input is an empty string
                batch_perplexity = [-1.0 if text == "" else perplex for text, perplex in zip(batch, batch_perplexity)]
                perplexities.extend(batch_perplexity)
                                
            except Exception as e:
                print(f"Error in batch {i//batch_size}: {str(e)}")
                perplexities.extend([1000.0] * len(batch))
        
        return perplexities

perplexity_calculator = PerplexityCalculator()

# ------ Tag Presence Reward ------
def tag_presence_reward(completions: List[dict]) -> List[float]:
    """Reward for presence of <reasoning> and <answer> tags"""
    rewards = []
    for completion in completions:
        content = completion[0]['content']
        has_reasoning = bool(re.search(r'<reasoning>.*?</reasoning>', content, re.DOTALL))
        has_answer = bool(re.search(r'<answer>.*?</answer>', content, re.DOTALL))
        reward = 0.5 * has_reasoning + 0.5 * has_answer
        rewards.append(reward)
    return rewards

# ------ Combined Reward Function ------
def combined_reward_func(
    prompts, completions, answer, **kwargs
) -> List[float]:
    # Extract generated responses
    responses = []
    valid_indices = []
    for idx, completion in enumerate(completions):
        try:
            generated_content = completion[0]['content'].strip()

            # Extract only the <answer> content using regex
            answer_match = re.search(r'<answer>(.*?)</answer>', generated_content, re.DOTALL)
            if answer_match:
                generated_content = answer_match.group(1).strip()
            else:
                responses.append("")  # Append empty string to maintain index consistency
                valid_indices.append(idx)
                continue
            
            # Skip if empty or just repeating the prompt
            user_prompt = prompts[idx][-1]['content']
            if not generated_content or generated_content == user_prompt:
                # print(f"generated_content: continue")
                responses.append("")  # Append empty string to maintain index consistency
                valid_indices.append(idx)
                continue
                
            responses.append(generated_content)
            # print(f"generated_content3: {generated_content}")
            valid_indices.append(idx)
        except (KeyError, IndexError):
            responses.append("")  # Append empty string to maintain index consistency
            valid_indices.append(idx)
            continue
    
    if not responses:
        return [-1.0] * len(completions)
    
    # Calculate rewards
    try:
        # Use raw answers without tag processing
        processed_answers = answer
        # print(f"processed_answers: {processed_answers[0]}")
        
        similarities = semantic_correctness(responses, [processed_answers[i] for i in valid_indices])
        perplexities = perplexity_calculator.calculate([processed_answers[i] for i in valid_indices])
        tag_rewards = tag_presence_reward([completions[i] for i in valid_indices])
    except Exception as e:
        print(f"Reward calculation error: {str(e)}")
        return [-1.0] * len(completions)
    
    # Convert to tensors with stability
    sim_scores = torch.nan_to_num(torch.tensor(similarities), nan=0.0)
    perplex_scores = torch.nan_to_num(torch.tensor(perplexities), nan=1000.0)
    tag_scores = torch.tensor(tag_rewards)
        
    # Perplexity reward calculation
    perplex_rewards = 1 / (perplex_scores / (perplex_scores.mean() + 1e-9))
    
    # Normalize with stability
    score_range = perplex_rewards.max() - perplex_rewards.min()
    if score_range < 1e-6:
        perplex_rewards_normalized = torch.ones_like(perplex_rewards) * 0.5
    else:
        perplex_rewards_normalized = (perplex_rewards - perplex_rewards.min()) / score_range
    
    # Combine scores with validation
    combined = [
        0.5 * sim.item() + 0.4 * pr.item() + 0.1 * tag.item()
        for sim, pr, tag in zip(sim_scores, perplex_rewards_normalized, tag_scores)
        if not torch.isnan(sim) and not torch.isnan(pr) and not torch.isnan(tag)
    ]
    
    # Map back to original indices
    final_rewards = [-1.0] * len(completions)
    for idx, reward in zip(valid_indices, combined):
        final_rewards[idx] = max(min(reward, 1.0), -1.0)  # Clamp between -1 and 1
    
    assert len(final_rewards) == len(completions), "Reward mapping error"
    return final_rewards

<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [7]:
checkpoint_repo = "matthewchung74/Qwen2.5_3B-GRPO-medical-reasoning-checkpoints"

per_device_train_batch_size = 1
gradient_accumulation_steps = 4

total_steps = len(train_dataset) // (per_device_train_batch_size * gradient_accumulation_steps)
num_checkpoints = 10

from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    use_vllm=True,  # use vLLM for fast inference!
    learning_rate=5e-6,
    adam_beta1=0.9,
    adam_beta2=0.99,
    weight_decay=0.1,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    optim="adamw_8bit",
    logging_steps=1,
    bf16=is_bfloat16_supported(),
    fp16=not is_bfloat16_supported(),
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    num_generations=5,  # Decrease if out of memory
    max_prompt_length=512,  # Updated to accommodate longer input prompts
    max_completion_length=312,
    max_steps=total_steps,
    save_steps=int(total_steps // num_checkpoints),
    max_grad_norm=0.1,
    report_to="none",  # Can use Weights & Biases
    output_dir=os.path.join(os.path.expanduser('~'), '.cache/huggingface', checkpoint_repo),
    save_strategy="steps",
    hub_model_id=checkpoint_repo,
    push_to_hub=True,
)

torch.distributed process group is initialized, but parallel_mode != ParallelMode.DISTRIBUTED. In order to use Torch DDP, launch your script with `python -m torch.distributed.launch


And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [8]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        combined_reward_func
    ],
    args = training_args,
    train_dataset = train_dataset,
)
trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 25,117 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 3,139
 "-____-"     Number of trainable parameters = 119,734,272


Step,Training Loss,reward,reward_std,completion_length,kl
1,-0.000000,0.018252,0.304542,248.475006,0.000000
2,-0.000000,0.114914,0.303148,260.425007,0.000000
3,0.000000,0.033591,0.272044,260.524998,0.000942
4,0.000100,-0.002997,0.346054,276.625000,0.001356
5,0.000000,-0.176495,0.277207,283.550007,0.001003
6,0.000000,0.018745,0.197320,257.125011,0.000858
7,0.000100,0.211401,0.378731,270.725002,0.001741
8,0.000100,-0.021427,0.334497,275.625004,0.001775
9,0.000100,-0.118577,0.204337,296.025009,0.001376
10,0.000000,0.178719,0.290878,258.724998,0.001228


WARNING 02-22 07:44:31 scheduler.py:1091] Input prompt (525 tokens) is too long and exceeds limit of 512
WARNING 02-22 07:44:31 scheduler.py:1091] Input prompt (525 tokens) is too long and exceeds limit of 512
WARNING 02-22 07:44:31 scheduler.py:1091] Input prompt (525 tokens) is too long and exceeds limit of 512
WARNING 02-22 07:44:31 scheduler.py:1091] Input prompt (525 tokens) is too long and exceeds limit of 512
WARNING 02-22 07:44:31 scheduler.py:1091] Input prompt (525 tokens) is too long and exceeds limit of 512
WARNING 02-22 08:26:27 scheduler.py:1091] Input prompt (521 tokens) is too long and exceeds limit of 512
WARNING 02-22 08:26:27 scheduler.py:1091] Input prompt (521 tokens) is too long and exceeds limit of 512
WARNING 02-22 08:26:27 scheduler.py:1091] Input prompt (521 tokens) is too long and exceeds limit of 512
WARNING 02-22 08:26:27 scheduler.py:1091] Input prompt (521 tokens) is too long and exceeds limit of 512
WARNING 02-22 08:26:27 scheduler.py:1091] Input prompt 

RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.cuda.FloatTensor instead (while checking arguments for embedding)

<a name="Inference"></a>
### Inference
Now let's try the model we just trained! First, let's first try the model without any GRPO trained:

And now with the LoRA we just trained with GRPO - we first save the LoRA first!

In [ ]:
model.save_lora("grpo_saved_lora")

Now we load the LoRA and test:

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "How many r's are in strawberry?"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 768,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Our reasoning model is much better - it's not always correct, since we only trained it for an hour or so - it'll be better if we extend the sequence length and train for longer!

<a name="Save"></a>
### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
import os

user_name = "matthewchung74"
model_name = "Qwen2.5_3B-GRPO-medical-reasoning"

if True:
    model.save_pretrained_merged(model_name, tokenizer, save_method = "merged_16bit",)    
    model.push_to_hub_merged(f"{user_name}/{model_name}", tokenizer, save_method="merged_16bit", token=os.environ["HF_TOKEN"])